## Imports

In [1]:
%pip install langchain
%pip install ipywidgets
%pip install langchain-openai


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import inspect_ai_integration
import code_from_inspect_ai

In [2]:
%env INSPECT_EVAL_MODEL=openai/gpt-4
%env INSPECT_MODEL_NAME=openai/gpt-4

# Create an instance of InspectChatModel with the specified model
inspect_model = code_from_inspect_ai.InspectChatModel()

# Create an instance of FactComparator with the InspectChatModel
comparator = inspect_ai_integration.FactComparator(inspect_model)


context = "The fox is brown. It runs quickly. The fox's best friend is Sally, which is a cat."
answer = "The fox is tan. It runs fast. Its best friend is a cat. She's named Sally."

# Run the asynchronous process_data method
result = await comparator(context, answer)

metrics = comparator.calculate_metrics(result["comparison_result"])


print("\nContext list:")
print(result["context_list"])

print("\nAnswer list:")
print(result["answer_list"])

print("\nComparison result:")
print(result["comparison_result"])

print("\nMetrics:")
print(f"Groundedness: {metrics['groundedness']:.2f}%")
print(f"Thoroughness: {metrics['thoroughness']:.2f}%")

env: INSPECT_EVAL_MODEL=openai/gpt-4
env: INSPECT_MODEL_NAME=openai/gpt-4

Context list:
<facts>
The fox is brown.
The fox runs quickly.
The fox's best friend is Sally.
Sally is a cat.
</facts>

Answer list:
<facts>
The fox is tan.
The fox runs fast.
The fox's best friend is a cat.
The cat's name is Sally.
</facts>

Comparison result:
facts_in_both=['The fox runs quickly.', "The fox's best friend is Sally.", 'Sally is a cat.'] facts_only_in_answer=['The fox is tan.'] facts_only_in_context=['The fox is brown.']

Metrics:
Groundedness: 75.00%
Thoroughness: 75.00%


In [3]:
# Example usage
cases = {
    'case1': {
        'input': 'The Sun is a medium-sized star. It\'s about 4.6 billion years old.',
        'target': 'The sun is approximately 4.6 billion years old. It\'s a mid-sized star.',
        'true_metrics': {'groundedness': 100, 'thoroughness': 100},
        'description': 'This is a basic use case with pronouns and mild rephrasing.'
    },
    'case2': {
        'input': 'The Sun, a medium-sized star, is located at the center of our Solar System and is approximately 4.6 billion years old.',
        'target': 'The sun is a mid-sized star which has existed for about 4.6 billion years.',
        'true_metrics': {'groundedness': 67, 'thoroughness': 100},
        'description': 'This is a basic use case with mild rephrasing.'
    },
    'case3': {
        'input': 'Sally is Rachel\'s cat.',
        'target': 'Sally is a cat. Rachel is her owner.',
        'true_metrics': {'groundedness': 100, 'thoroughness': 100},  
        'description': 'This case involves simple restructuring and clarification.'
    },
    'case4': {
        'input': 'Sally is larger than Stan.',
        'target': 'Stan is smaller than Sally.',
        'true_metrics': {'groundedness': 100, 'thoroughness': 100}, 
        'description': 'This case demonstrates a change in comparative perspective.'
    },
    'case5': {
        'input': 'the average temperature today is 20 degrees celsius.',
        'target': 'the mean temperature today is 68 degrees fahrenheit.',
        'true_metrics': {'groundedness': 100, 'thoroughness': 100},  
        'description': 'This case involves unit conversion and synonym use.'
    },
    'case6': {
        'input': 'the average temperature today is 20 degrees celsius.',
        'target': 'the average temperature today is 50 degrees celsius.',
        'true_metrics': {'groundedness': 0, 'thoroughness': 0},  
        'description': 'This case involves unit conversion and synonym use.'
    },
    'case7': {
        'input': 'The company has an ATO now, so they have been sanctioned by the government and you can work with them.', 
        'target':  'The company has been sanctioned by the government in response to recent lawbreaking activity.' , 
        'true_metrics': {'groundedness': 0, 'thoroughness': 0},  # Contextual misuse
        'description': 'This case uses "sanctioned" in a way that highlights its dual meaning: approved or penalized.'
    },
    # Add more cases as needed
}


%env INSPECT_EVAL_MODEL=openai/gpt-4
%env INSPECT_MODEL_NAME=openai/gpt-4


df = inspect_ai_integration.compare_metrics(cases)

env: INSPECT_EVAL_MODEL=openai/gpt-4
env: INSPECT_MODEL_NAME=openai/gpt-4


In [4]:
df

,Case,Input Statement,Target Statement,Description,Groundedness (True),Thoroughness (True),Groundedness (Model),Thoroughness (Model),Context List,Answer List,Facts in Both,Facts Only in Answer,Facts Only in Context,Model Error
0,case1,The Sun is a medium-sized star. It's about 4.6...,The sun is approximately 4.6 billion years old...,This is a basic use case with pronouns and mil...,100,100,100.000000,100.0,<facts>\nThe sun is approximately 4.6 billion ...,<facts>\nThe Sun is a medium-sized star.\nThe ...,[The sun is approximately 4.6 billion years ol...,[],[],None
1,case2,"The Sun, a medium-sized star, is located at th...",The sun is a mid-sized star which has existed ...,This is a basic use case with mild rephrasing.,67,100,66.666667,100.0,<facts>\nThe sun is a mid-sized star.\nThe sun...,<facts>\nThe Sun is a medium-sized star.\nThe ...,"[The sun is a mid-sized star., The sun has exi...",[The Sun is located at the center of our Solar...,[],None
2,case3,Sally is Rachel's cat.,Sally is a cat. Rachel is her owner.,This case involves simple restructuring and cl...,100,100,100.000000,100.0,<facts>\nSally is a cat.\nRachel is Sally's ow...,<facts>\nSally is Rachel's cat.\n</facts>,"[Sally is a cat., Rachel is Sally's owner.]",[],[],None
3,case4,Sally is larger than Stan.,Stan is smaller than Sally.,This case demonstrates a change in comparative...,100,100,100.000000,100.0,<facts>\nStan is smaller than Sally.\n</facts>,<facts>\nSally is larger than Stan.\n</facts>,"[Stan is smaller than Sally., Sally is larger ...",[],[],None
4,case5,the average temperature today is 20 degrees ce...,the mean temperature today is 68 degrees fahre...,This case involves unit conversion and synonym...,100,100,100.000000,100.0,<facts>\nThe mean temperature today is 68 degr...,<facts>\nThe average temperature today is 20 d...,[The mean/average temperature today is 68 degr...,[],[],None
5,case6,the average temperature today is 20 degrees ce...,the average temperature today is 50 degrees ce...,This case involves unit conversion and synonym...,0,0,0.000000,0.0,<facts>\nThe average temperature today is 50 d...,<facts>\nThe average temperature today is 20 d...,[],[The average temperature today is 20 degrees c...,[The average temperature today is 50 degrees c...,None
6,case7,"The company has an ATO now, so they have been ...",The company has been sanctioned by the governm...,"This case uses ""sanctioned"" in a way that high...",0,0,33.333333,50.0,<facts>\nThe company has been sanctioned by th...,<facts>\nThe company has an ATO now.\nThe comp...,[The company has been sanctioned by the govern...,"[The company has an ATO now, You can work with...",[The sanction was in response to recent lawbre...,None


In [4]:
from inspect_ai import eval, Task, task

%env INSPECT_EVAL_MODEL=openai/gpt-4
%env INSPECT_MODEL_NAME=openai/gpt-4

eval(inspect_ai_integration.my_eval(), model="openai/gpt-3.5-turbo")

env: INSPECT_EVAL_MODEL=openai/gpt-4
env: INSPECT_MODEL_NAME=openai/gpt-4


Output()

value={'groundedness': 50.0, 'thoroughness': 100.0} answer=None explanation="{'context_list': '<facts>\\nThe sun is
estimated to be about 4.6 billion years old.\\n</facts>', 'answer_list': '<facts>\\nThe sun is approximately 4.6 
billion years old.\\nThe sun is a mid-sized star.\\n</facts>', 'comparison_result': 
ComparisonResult(facts_in_both=['The sun is estimated to be about 4.6 billion years old.'], 
facts_only_in_answer=['The sun is a mid-sized star.'], facts_only_in_context=[])}\nModel Output: The sun is 
estimated to be about 4.6 billion years old." metadata=None